In [27]:
### Pacotes necessarios:
# install.packages("e1071")
# install.packages("caret")
# install.packages("kernlab")
library("kernlab")
library("caret")
library(mlbench)


In [28]:
#carregando a base de dados
dados <- read.csv("databases/1 - Cancer de Mama - Dados.csv")

#removendo Id
dados$Id <- NULL
head(dados)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5,1,1,1,2,1,3,1,1,benign
2,5,4,4,5,7,10,3,2,1,benign
3,3,1,1,1,2,2,3,1,1,benign
4,6,8,8,1,3,4,3,7,1,benign
5,4,1,1,3,2,1,3,1,1,benign
6,8,10,10,8,7,10,9,7,1,malignant


In [29]:
#separando a base em 80% 20%
set.seed(1912)
indices <- createDataPartition(dados$Class, p = 0.80, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]


In [30]:
#criando modelo usando svm Radial
set.seed(1912)
svm <- train(Class ~ ., data = treino, method = "svmRadial")
svm

Support Vector Machines with Radial Basis Function Kernel 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.9396639  0.8716127
  0.50  0.9461022  0.8845589
  1.00  0.9513385  0.8951586

Tuning parameter 'sigma' was held constant at a value of 0.7308222
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.7308222 and C = 1.

In [31]:
#aplicando o modelo a base de teste e gerando matriz de confusao
predict_svm <- predict(svm, teste)

confusionMatrix(predict_svm, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         3
  malignant      4        45
                                         
               Accuracy : 0.9496         
                 95% CI : (0.899, 0.9795)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.8892         
                                         
 Mcnemar's Test P-Value : 1              
                                         
            Sensitivity : 0.9560         
            Specificity : 0.9375         
         Pos Pred Value : 0.9667         
         Neg Pred Value : 0.9184         
             Prevalence : 0.6547         
         Detection Rate : 0.6259         
   Detection Prevalence : 0.6475         
      Balanced Accuracy : 0.9468         
                                         
       'Positive' Class : benign         
                   

In [32]:
#Cross validation no modelo svm
ctrl <- trainControl(method = "cv", number = 10)

set.seed(1912)
svm <- train(Class ~ ., data = treino, method = "svmRadial", trControl = ctrl)
svm


Support Vector Machines with Radial Basis Function Kernel 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 504, 503, 505, 504, 504, 505, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.9390841  0.8726515
  0.50  0.9516815  0.8979117
  1.00  0.9552854  0.9049221

Tuning parameter 'sigma' was held constant at a value of 0.7308222
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.7308222 and C = 1.

In [33]:
#aplicando o modelo a base de teste e gerando matriz de confusao com cross validation
predict_svm <- predict(svm, teste)

confusionMatrix(predict_svm, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         3
  malignant      4        45
                                         
               Accuracy : 0.9496         
                 95% CI : (0.899, 0.9795)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.8892         
                                         
 Mcnemar's Test P-Value : 1              
                                         
            Sensitivity : 0.9560         
            Specificity : 0.9375         
         Pos Pred Value : 0.9667         
         Neg Pred Value : 0.9184         
             Prevalence : 0.6547         
         Detection Rate : 0.6259         
   Detection Prevalence : 0.6475         
      Balanced Accuracy : 0.9468         
                                         
       'Positive' Class : benign         
                   

In [34]:
#testando varios valores de custo e sigma
tune_grid <- expand.grid(C = c(1, 2, 10, 50, 100), sigma = c(0.01, 0.015, 0.2))

set.seed(1912)
svm <- train(Class ~ ., data = treino, method = "svmRadial", trControl = ctrl, tuneGrid = tune_grid)
svm

Support Vector Machines with Radial Basis Function Kernel 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 504, 503, 505, 504, 504, 505, ... 
Resampling results across tuning parameters:

  C    sigma  Accuracy   Kappa    
    1  0.010  0.9642174  0.9220457
    1  0.015  0.9624316  0.9181159
    1  0.200  0.9605485  0.9142396
    2  0.010  0.9606459  0.9139930
    2  0.015  0.9624316  0.9176311
    2  0.200  0.9587315  0.9094341
   10  0.010  0.9623679  0.9173119
   10  0.015  0.9659717  0.9251139
   10  0.200  0.9445386  0.8777963
   50  0.010  0.9641860  0.9210483
   50  0.015  0.9641860  0.9210483
   50  0.200  0.9428480  0.8736058
  100  0.010  0.9641860  0.9210483
  100  0.015  0.9569782  0.9049272
  100  0.200  0.9428480  0.8736058

Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.015 and C = 10.

In [35]:
#aplicando o modelo a base de teste e gerando matriz de confusao
predict_svm <- predict(svm, teste)

confusionMatrix(predict_svm, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        90         4
  malignant      1        44
                                          
               Accuracy : 0.964           
                 95% CI : (0.9181, 0.9882)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9193          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.9890          
            Specificity : 0.9167          
         Pos Pred Value : 0.9574          
         Neg Pred Value : 0.9778          
             Prevalence : 0.6547          
         Detection Rate : 0.6475          
   Detection Prevalence : 0.6763          
      Balanced Accuracy : 0.9528          
                                          
       'Positive' Class : benign          

In [36]:
#predicao de novos casos
dados_novos_casos <- read.csv("databases/1 - Cancer de Mama - Dados - Novos Casos.csv")

dados_novos_casos$Id <- NULL
View(dados_novos_casos)
dados_novos_casos$Class <- NULL

predict_svm <- predict(svm, dados_novos_casos)

resultado <- cbind(dados_novos_casos, predict_svm)
View(resultado)

Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2,1,5,1,7,1,3,10,5,?
2,3,4,5,3,1,3,2,1,?
7,1,1,5,2,2,2,1,1,?


Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,predict_svm
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,1,5,1,7,1,3,10,5,malignant
2,3,4,5,3,1,3,2,1,benign
7,1,1,5,2,2,2,1,1,benign
